In [2]:
%pylab
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage
import scipy
import pandas as pd
import seaborn as sns
path =  (os.path.abspath(os.path.join(r'.', os.pardir)))
import sys
sys.path.append(path)
from nuclei_segmentation import otsu as ot
from nuclei_segmentation import dicescore as dsc
from PIL import Image
import os
import os.path

Using matplotlib backend: <object object at 0x00000268CC4740D0>
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


c:\Users\Helen Jade\anaconda3\envs\dataanalysis0104_Env\lib\site-packages\IPython\core\magics\pylab.py:162: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [5]:
# Set ground truth images from N2DH-GOWT1 dataset and N2DL-HeLa dataset to 0 and 1

def binarize(x):
    img = x.copy()

    for o in np.ndindex(img.shape):
        if img[o] > 0: 
           img[o] = 1
        else:
            img[o] = 0
    
    return img

## NIH3T3 Dataset

In [6]:
# Load raw images of NIH3T3-dataset

img_NIH3T3 = []
path = os.path.join(os.path.abspath(os.path.join(r'.', os.pardir)),r"data/Otsu_data/NIH3T3/img")
for f in os.listdir(path):
    img_NIH3T3.append(imread(os.path.join(path , f)))

# Load ground truth images of NIH3T3-dataset
 
gt_NIH3T3 = []
path = os.path.join(os.path.abspath(os.path.join(r'.', os.pardir)),r"data/Otsu_data/NIH3T3/gt")
for f in os.listdir(path):
    gt_NIH3T3.append(imread(os.path.join(path , f)))

In [7]:
# Make a copy of all images of each dataset / use as "no preprocessed images"
copy_img_NIH3T3 = img_NIH3T3.copy()

In [7]:
print(gt_NIH3T3)

[array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0

In [8]:
print(img_NIH3T3)

[array([[0.14509805, 0.15294118, 0.14901961, ..., 0.07843138, 0.07843138,
        0.07450981],
       [0.16078432, 0.14901961, 0.14901961, ..., 0.07843138, 0.08235294,
        0.08235294],
       [0.14901961, 0.14901961, 0.15686275, ..., 0.08235294, 0.08235294,
        0.07843138],
       ...,
       [0.08627451, 0.08235294, 0.08627451, ..., 0.09411765, 0.09019608,
        0.09019608],
       [0.08627451, 0.09019608, 0.08627451, ..., 0.09411765, 0.09019608,
        0.09411765],
       [0.08627451, 0.08235294, 0.08627451, ..., 0.09411765, 0.09411765,
        0.09019608]], dtype=float32), array([[0.08235294, 0.08235294, 0.08235294, ..., 0.12941177, 0.1254902 ,
        0.13333334],
       [0.07843138, 0.08235294, 0.08627451, ..., 0.12941177, 0.13333334,
        0.1254902 ],
       [0.08235294, 0.08627451, 0.08235294, ..., 0.12941177, 0.12941177,
        0.13333334],
       ...,
       [0.13333334, 0.13333334, 0.12156863, ..., 0.08627451, 0.08235294,
        0.08235294],
       [0.14901961

#### Median filter

In [8]:
# Apply median filter to all images of each dataset
# mit kleinerern Filtern läuft es schneller durch
# filter größen die man möchte
filter = [3, 4, 5, 10, 15, 20]
median_img_NIH3T3 = []
for j in range (0, len(filter)):
    
    for i in range(0, len(copy_img_NIH3T3)):
        median_img_NIH3T3.append(scipy.ndimage.median_filter(copy_img_NIH3T3[i] , size= filter[j]))
    print("filtersize done")    

filtersize done
filtersize done
filtersize done
filtersize done
filtersize done
filtersize done


In [44]:
#mit 2 D array geht so aber-> otsu nicht mehr möglich


# Apply median filter to all images of each dataset
# mit kleinerern Filtern läuft es schneller durch
# filter größen die man möchte
#filter = [3, 4, 5]
#median3_img_NIH3T3 = []
#median4_img_NIH3T3 = []
#median5_img_NIH3T3 = []

#for j in range (0, len(filter)):
   # median_img_NIH3T3 = []
    #for i in range(0, len(copy_img_NIH3T3)):
    #    median3_img_NIH3T3.append(scipy.ndimage.median_filter(copy_img_NIH3T3[i] , size= filter[0]))
    #    median4_img_NIH3T3.append(scipy.ndimage.median_filter(copy_img_NIH3T3[i] , size= filter[1]))
    #    median5_img_NIH3T3.append(scipy.ndimage.median_filter(copy_img_NIH3T3[i] , size= filter[2]))
    #median_img_NIH3T3.append(median3_img_NIH3T3) 
    #median_img_NIH3T3.append(median4_img_NIH3T3)
    #median_img_NIH3T3.append(median5_img_NIH3T3)

    #print("filtersize done")  

filtersize done
filtersize done
filtersize done


#### gaussian filter

In [ ]:
# Apply gaussian filter to all images of each dataset
#filter größen die mach möchte 
filter_g = [3, 4, 5, 10, 15, 20]
gauss_img_NIH3T3 = []

for j in range (0, len(filter_g)):

    for i in range(0,len(copy_img_NIH3T3)):
        gauss_img_NIH3T3.append(scipy.ndimage.gaussian_filter(copy_img_NIH3T3[i] , filter_g[j]))
    print("filtersize done")
        


## Otsu thresholding

In [9]:
# Apply otsu thresholding to all images of each dataset - median filter
otsu_median_img_NIH3T3 = []
for i in range(0,len(median_img_NIH3T3)):
    otsu_median_img_NIH3T3.append(ot.otsu_thresholding(median_img_NIH3T3 [i] , 256))




In [48]:
#otsu median mit 2D array-> nicht möglich

#otsu_median_img_NIH3T3 = []
#otsu3_median_img_NIH3T3 = []
#otsu4_median_img_NIH3T3 = []
#otsu5_median_img_NIH3T3 = []

#for i in range(0,len(median_img_NIH3T3)):
    #otsu3_median_img_NIH3T3.append(ot.otsu_thresholding(median_img_NIH3T3 [0] , 256))
    #otsu4_median_img_NIH3T3.append(ot.otsu_thresholding(median_img_NIH3T3 [1] , 256))
    #otsu5_median_img_NIH3T3.append(ot.otsu_thresholding(median_img_NIH3T3 [2] , 256))
#otsu_median_img_NIH3T3.append(otsu3_median_img_NIH3T3)
#otsu_median_img_NIH3T3.append(otsu4_median_img_NIH3T3)
#otsu_median_img_NIH3T3.append(otsu5_median_img_NIH3T3)



AttributeError: 'list' object has no attribute 'flatten'

In [ ]:
# Apply otsu thresholding to all images of each dataset - gaussian filter
otsu_gauss_img_NIH3T3 = []
for i in range(0,len(gauss_img_NIH3T3)):
    otsu_gauss_img_NIH3T3.append(ot.otsu_thresholding(gauss_img_NIH3T3[i] , 256))


In [11]:
gt6_NIH3T3 = []

path = os.path.join(os.path.abspath(os.path.join(r'.', os.pardir)),r"data/Otsu_data/NIH3T3/gt")
for f in os.listdir(path):
    gt6_NIH3T3.append(imread(os.path.join(path , f)))
    gt6_NIH3T3.append(imread(os.path.join(path , f)))
    gt6_NIH3T3.append(imread(os.path.join(path , f)))
    gt6_NIH3T3.append(imread(os.path.join(path , f)))
    gt6_NIH3T3.append(imread(os.path.join(path , f)))
    gt6_NIH3T3.append(imread(os.path.join(path , f)))

print(len(gt6_NIH3T3))

108


## Dice score

In [16]:
# Dice score - NIH3T3-dataset
## median filter
dice_otsu_median_img_NIH3T3 = []

for i in range(0, len(otsu_median_img_NIH3T3)):
  dice_otsu_median_img_NIH3T3.append(dsc.dice(otsu_median_img_NIH3T3[i] , gt6_NIH3T3[i]))

print(dice_otsu_median_img_NIH3T3)

### Es gibt mir zwar 18 mal 6 werte aus (also für die 6 filtersizes die ich gewählt habe)
# aber die werte stimmen nicht... wenn man sie mit "window_preprocessing_short.ipynb" vergleicht wo man die richtige
# dice score werte sieht für NIH3T3 für eine filtersize von 5 was hier also bei reihe  7 anfangen würde...
# sieht so aus als würde er nur das erste bild korrekt dice scoren (0.913...) und danach kommt nur noch doofes zeug raus...

##dice_otsu_median_img_NIH3T3 = []
#for i in range (0, len (gt6_NIH3T3)):
   #for j in range( 0, len(otsu_median_img_NIH3T3)):
     #dice_otsu_median_img_NIH3T3.append(dsc.dice(otsu_median_img_NIH3T3[j] , gt6_NIH3T3[i]))
#print(dice_otsu_median_img_NIH3T3)



[0.9131440698604878, 0.1693720666222276, 0.19193484084407544, 0.19049441590949104, 0.18228644043113887, 0.12653026281996405, 0.15904731678372822, 0.20306007886801206, 0.0728409001391697, 0.1201484259306182, 0.010371552313653977, 0.14662280040678508, 0.0070859996878414235, 0.18209939592027316, 0.2522687631494058, 0.05925468713833809, 0.03465325010093481, 0.2453728234876724, 0.16302326970161435, 0.14880459141395794, 0.19185061418459962, 0.739506084579049, 0.23186058141810353, 0.148542193426043, 0.17897354736604115, 0.20085775152186255, 0.05124583257515243, 0.12240642324399421, 0.028614258797053236, 0.22406422653087826, 0.006922774141450211, 0.19481097420180943, 0.22529109844044665, 0.011847284871289223, 0.07132610294265382, 0.19004662302300782, 0.1859963683725175, 0.15823245919905873, 0.23112564731534477, 0.2303489072427151, 0.22399109379348733, 0.19518577184050048, 0.22923682473610932, 0.6973465117594204, 0.03456887209985419, 0.11962055665589492, 0.002028496053604087, 0.2683351633249737

In [38]:
print(len(gt_NIH3T3)) #<- muss irgendwie auch 108 lang, bild muss sich quasi immer wiederhoen
print(len(gt6_NIH3T3))
print(len(median_img_NIH3T3))

18
6
108


In [48]:
print(otsu_median_img_NIH3T3)

[array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0

In [ ]:
# Dice score - N2DH-GOWT1-dataset
## gaussian filter

dice_otsu_gauss_img_N2DH_GOWT1 = []
for j in range(0,len(otsu_gauss_img_N2DH_GOWT1)):
    dice_otsu_gauss_img_N2DH_GOWT1.append(dsc.dice(otsu_gauss_img_N2DH_GOWT1[j] , binary_gt_N2DH_GOWT1[j]))

print(dice_otsu_gauss_img_N2DH_GOWT1)

NameError: name 'otsu_gauss_img_N2DH_GOWT1' is not defined